In [33]:
import tensorflow as tf
import datetime
import os

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


### Beginners
---

In [34]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [35]:
x_train, x_test = x_train / 255., x_test / 255.

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10),
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [5]:
predictions = model(x_train[:5])
softmaxes = tf.nn.softmax(predictions)
loss = loss_fn(y_train[:5], predictions)

print("softmax: {}\nloss: {}".format(softmaxes.numpy(), loss.numpy()))

softmax: [[0.06551202 0.10176755 0.10221194 0.09102646 0.20914747 0.12792437
  0.13070227 0.02745285 0.05136614 0.09288896]
 [0.11282706 0.09303643 0.06275751 0.13158062 0.21336561 0.11631046
  0.10072237 0.03072745 0.09185615 0.04681639]
 [0.1524573  0.1176155  0.09211686 0.09854499 0.07904945 0.13464382
  0.09854833 0.07649301 0.0567124  0.09381834]
 [0.08792618 0.09525195 0.0839536  0.13422942 0.13698672 0.16202153
  0.08478946 0.0313777  0.10290521 0.08055819]
 [0.10210298 0.07720765 0.07102622 0.12999284 0.15048036 0.12593955
  0.11515926 0.03752364 0.06432055 0.12624691]]
loss: 2.239328384399414


In [10]:
model.compile(
    optimizer='adam',
    loss=loss_fn,
    metrics=['accuracy']
)

In [11]:
log_dir = os.path.join('log', 'fit', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [12]:
model.fit(
    x_train, y_train,
    epochs=5,
    validation_data=(x_test, y_test),
    callbacks=[tensorboard_callback],
)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2944 - accuracy: 0.9138 - val_loss: 0.1376 - val_accuracy: 0.9589
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1414 - accuracy: 0.9580 - val_loss: 0.1048 - val_accuracy: 0.9693
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1052 - accuracy: 0.9681 - val_loss: 0.0832 - val_accuracy: 0.9744
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0856 - accuracy: 0.9736 - val_loss: 0.0781 - val_accuracy: 0.9752
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0735 - accuracy: 0.9766 - val_loss: 0.0675 - val_accuracy: 0.9772


In [20]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 0s - loss: 0.0702 - accuracy: 0.9784


[0.07015503197908401, 0.9783999919891357]

### Experts
---

In [15]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [16]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255., x_test / 255.
x_train, x_test = x_train[..., tf.newaxis], x_test[..., tf.newaxis]

In [17]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [20]:
class NewModel(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)
        
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.d2(x)
        return x
        
model = NewModel()

In [22]:
loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [23]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_obj(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)
    
@tf.function
def test_step(images, labels):
    predictions = model(images, training=True)
    loss = loss_obj(labels, predictions)
    test_loss(loss)
    test_accuracy(labels, predictions)
    

In [30]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = os.path.join("logs", "gradient_tape", current_time, "train")
test_log_dir = os.path.join("logs", "gradient_tape", current_time, "test")

train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [31]:
EPOCHS = 5

for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    for images, labels in train_ds:
        train_step(images, labels)
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)
                
    for images, labels in test_ds:
        test_step(images, labels)
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
        
        
    print("EPOCH {} --- Loss: {:.3f}, Acc: {:.3f}, Test Loss: {:.3f}, Test Acc: {:.3f}".format(
        epoch + 1, train_loss.result(), train_accuracy.result(), test_loss.result(), test_accuracy.result(),
    ))

EPOCH 1 --- Loss: 0.001, Acc: 1.000, Test Loss: 0.114, Test Acc: 0.983
EPOCH 2 --- Loss: 0.003, Acc: 0.999, Test Loss: 0.106, Test Acc: 0.984
EPOCH 3 --- Loss: 0.001, Acc: 1.000, Test Loss: 0.107, Test Acc: 0.983
EPOCH 4 --- Loss: 0.002, Acc: 0.999, Test Loss: 0.121, Test Acc: 0.982
EPOCH 5 --- Loss: 0.002, Acc: 1.000, Test Loss: 0.109, Test Acc: 0.984
